In [9]:
import numpy as np 
from matplotlib import pyplot as plt 
from mpl_toolkits import mplot3d
from copy import copy

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 10]

### Non Linear upwind scheme
The PDE to be solved is 
$$ u_{t} + uu_{x} = \nu u_{xx} \\
   u(x, 0) = x \\ u(0, t) = 0 \\ u(1, t) = 1$$
   Here the value of $\nu = 0.5$
   
Using Quasi-Linear Discretization, we get 

$$ {u^{(n+1)^{(k+1)}} - u^{n}_{j} \over \partial(t)} + \frac{1}{2}( u^{n}_{j}(\frac{u^n_{j+1} - u^{n}_{j-1}}{\partial{x}} ) +  u^{(n+1)^{(k+1)}}_{j}(\frac{u^{(n+1)^{(k+1)}}_{j+1} - u^{(n+1)^{(k+1)}}_{j-1}}{\partial{x}} ) ) = \frac{\nu}{2}\big[\frac{u^{(n+1)^{(k+1)}}_{j+1} - 2u^{(n+1)^{(k+1)}}_{j} + u^{(n+1)^{(k+1)}}_{j-1}}{\partial{x}^2} + \frac{u^n_{j+1} - 2u^n_{j} + u^{n}_{j-1}}{\partial{x}^2}\big]$$

Converting it in the form of  
$$ U^{{n+1}^{k+1}}_{j} = U^{{n+1}^{k}}_{j} + \Delta U ^{n+1}_{j}\\ A_i \Delta U ^{n+1}_{j-1} + B_i \Delta U ^{n+1}_{j} + C_i \Delta U ^{n+1}_{j+1} = D_i$$
  Take $$ c = {\partial{t} \over 2\partial{x}} \\
          r = \nu \frac{\partial{t}}{2\partial{x}^2}$$

In [128]:
def ThomasAlgorithm(a, b, c, d, n):
    c_dash = np.zeros(n-1)
    d_dash = np.zeros(n-1)
    c_dash[0] = c[0]/b[0]
    d_dash[0] = d[0]/b[0]
    for itr in range(1, n-1):
        c_dash[itr] = c[itr] / (b[itr] - a[itr] * c_dash[itr-1])
        d_dash[itr] = (d[itr] - a[itr]*d_dash[itr-1]) / (b[itr] - a[itr] * c_dash[itr-1])
    
    y = np.zeros(n-1)
    y[n-2] = d_dash[n-2]
    
    for itr in reversed(range(n-2)):
        y[itr] = d_dash[itr] - c_dash[itr] * y[itr+1]
    
    return y

In [129]:
def A(u, u_prev, r, c, i):
    temp = r + c*u[i]
    return temp
def B(u, u_prev, r, c, j):
    temp = c*(u[j-1] - u[j+1]) - 2*r
    return temp
def C(u, u_prev, r, c, i):
    temp = r - c*(u[i])
    return temp
def D(u, u_prev, r, c, j):
    term1 = u[j] - u_prev[j]
    term2 = c*(u_prev[j]*(u_prev[j+1] - u_prev[j-1]) + u[j]*(u[j+1] - u[j-1]))
    term3 = r*(u[j+1] - 2*u[j] + u[j-1] +  u_prev[j+1] - 2*u_prev[j] + u_prev[j-1])
    temp = term1 + term2 - term3
    return temp 

In [136]:
def initialize(x0, xn, steps):
    u = np.linspace(x0, xn, steps + 1)
    return u

def Solver(x0, xn, space_steps, t_steps, r, const, epsilon):
    u = initialize(x0, xn, space_steps)
    Solution = np.array([[]])
    Solution = np.append(Solution, np.reshape(u, (1, u.shape[0])), axis = 1)
    delta_u = np.zeros(u.shape[0])
    for i in range(t_steps):
        flag = 1
        u_prev = np.copy(u)
        count = 0 
        while(np.amax(delta_u) > epsilon or flag ==1 or count==100):
            if (flag == 1):
                flag = 0
                
            else:
                u = u + delta_u
            a = [A(u, u_prev, r, const, i) for i in range(space_steps-1)]
            b = [B(u, u_prev, r, const, i) for i in range(space_steps-1)]
            c = [C(u, u_prev, r, const, i) for i in range(space_steps-1)]
            d = [D(u, u_prev, r, const, i) for i in range(space_steps-1)]
            delta_u[1: -1] = ThomasAlgorithm(a, b, c, d, space_steps)
            count+=1
        
        temp = np.reshape(u, (1, u.shape[0]))
        Solution = np.append(Solution, temp, axis=0)
    return Solution

In [137]:
x0 = 0 
xn = 1
space_steps = 10
time_steps = 20
nu = 0.5
r = nu*(space_steps**2/time_steps)/2
const = space_steps/(2*time_steps)

In [138]:
Solution = Solver(x0, xn, space_steps, time_steps, r, const, epsilon = 0.01)
print(Solution)
x = np.linspace(x0, xn, num=space_steps+1)
for i in range(Solution.shape[0]):
      plt.plot(x, Solution[i])
plt.show()
plt.savefig("Plot.png")

KeyboardInterrupt: 